In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical 

In [2]:
DATADIR = "D:/THESIS/Spectrogram/"

In [3]:
categories = os.listdir(DATADIR)
categories

['air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music']

In [4]:
data = []

def create_dataset():
    for category in categories:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = categories.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img)) ##,cv2.IMREAD_GRAYSCALE)  # convert to array
                #rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array =img_array[34:252, 53:389]
                #print(new_array.shape)
                rsz=cv2.resize(new_array, (224, 224))
                #print(rsz.shape)
                data.append([rsz, class_num])  # add this to our training_data
                #plt.axis('off')
                #plt.imshow(rsz)
                #plt.savefig("ragresize.png")
                #break
            except Exception as e:  # in the interest in keeping the output clean...
                pass
    return data



In [5]:
data = create_dataset()

print(len(data))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 52.99it/s]

8732


In [6]:
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)

In [7]:
import pickle

In [8]:
pickle_out = open("X_DATA_224x224.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("Y_DATA_224x224.pickle","wb")
pickle.dump(Y,pickle_out)
pickle_out.close()


In [7]:
X = np.array(X)
Y = np.array(Y)

In [9]:
X.shape

(8732, 100, 100, 3)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_data, X_test, Y_data, Y_test = train_test_split(X, Y,test_size=0.10, random_state = 42,stratify=Y)

In [12]:
X_train,X_val, Y_train, Y_val = train_test_split(X_data, Y_data,test_size=0.20, random_state = 42)

In [13]:
def normalizaze_data(X):
    X_data = []
    for data in X:
        data = data / 255
        X_data.append(data)
    return np.array(X_data)

In [14]:
%%time
x_train = normalizaze_data(X_train)
x_val = normalizaze_data(X_val)
x_test = normalizaze_data(X_test)

Wall time: 2.21 s


In [15]:
x_train = x_train.reshape(-1, 100, 100, 3)
x_val = x_val.reshape(-1, 100, 100, 3)
x_test = x_test.reshape(-1, 100, 100, 3)

In [16]:
%%time
y_train =to_categorical(Y_train)
y_val = to_categorical(Y_val)
y_test =to_categorical(Y_test)

Wall time: 53.7 ms


In [17]:
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_val.shape: {x_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (6286, 100, 100, 3)
y_train.shape: (6286, 10)
x_val.shape: (1572, 100, 100, 3)
y_val.shape: (1572, 10)
x_test.shape: (874, 100, 100, 3)
y_test.shape: (874, 10)


In [18]:
pickle_out = open("x_train_200X200.pickle","wb")
pickle.dump(x_train,pickle_out)
pickle_out.close()

pickle_out = open("y_train_200X200.pickle","wb")
pickle.dump(y_train,pickle_out)
pickle_out.close()

pickle_out = open("x_val_200X200.pickle","wb")
pickle.dump(x_val,pickle_out)
pickle_out.close()

pickle_out = open("y_val_200X200.pickle","wb")
pickle.dump(y_val,pickle_out)
pickle_out.close()

pickle_out = open("x_test_200X200.pickle","wb")
pickle.dump(x_test,pickle_out)
pickle_out.close()

pickle_out = open("y_test_200X200.pickle","wb")
pickle.dump(y_test,pickle_out)
pickle_out.close()

In [19]:
#importing other required libraries
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical
from keras import losses, models, optimizers
import time
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

In [20]:
x_train = pickle.load(open("x_train_200X200.pickle",'rb'))
y_train = pickle.load(open("y_train_200X200.pickle",'rb'))

x_val = pickle.load(open("x_val_200X200.pickle",'rb'))
y_val = pickle.load(open("y_val_200X200.pickle",'rb'))

x_test = pickle.load(open("x_test_200X200.pickle",'rb'))
y_test = pickle.load(open("y_test_200X200.pickle",'rb'))

In [21]:
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_val.shape: {x_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"x_val.shape: {x_test.shape}")
print(f"y_val.shape: {y_test.shape}")

x_train.shape: (6286, 100, 100, 3)
y_train.shape: (6286, 10)
x_val.shape: (1572, 100, 100, 3)
y_val.shape: (1572, 10)
x_val.shape: (874, 100, 100, 3)
y_val.shape: (874, 10)


In [22]:
#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

val_generator = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

test_generator = ImageDataGenerator(rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [23]:
#Learning Rate Annealer

lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)

In [24]:
##!pip install keras_efficientnets

from tensorflow.keras.applications import EfficientNetB0

from tensorflow.keras.applications import EfficientNetB1

In [25]:
#Defining the model
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=x_train.shape[1:],classes=y_train.shape[1])

#Adding the final layers to the above base models where the actual classification is done in the dense layers

model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

#Model summary
model.summary()

#Adding the Dense layers along with activation and batch normalization
#model.add(Dense(1024,activation=('relu'),input_dim=512))
#model.add(Dropout(.5))
#model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
#model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model.add(Dense(10,activation=('softmax'))) 

#Checking the final model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 4, 4, 1280)        4049571   
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0         
Total params: 4,049,571
Trainable params: 4,007,548
Non-trainable params: 42,023
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 4, 4, 1280)        4049571   
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               5243136   
_____________________

In [26]:
tf.optimizers.Adam()

In [28]:
#Defining the parameters
batch_size= 64
epochs=150
learn_rate=.001
MAX_PATIENT=5

In [29]:
#from tensorflow.optimizers import SGD
#sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

In [30]:
%%time
opt = tf.optimizers.Adam(0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('efficientnetB0_100x100.h5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=MAX_PATIENT)
ReduceLR = ReduceLROnPlateau(monitpr = 'val_loss',factor=0.1,patience=3, verbose=1)
callbacks_list = [checkpoint,ReduceLR, early]

print("[INFO] Training Start---------please Wait")
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs, 
                    validation_data =(x_val, y_val),
                    verbose = 1,
                   callbacks = callbacks_list)
print("[INFO] Training Finished!!!")

[INFO] Training Start---------please Wait
Epoch 1/150
99/99 [==============================] - 322s 3s/step - loss: 0.9278 - accuracy: 0.6995 - val_loss: 4.2113 - val_accuracy: 0.1005

Epoch 00001: val_loss improved from inf to 4.21131, saving model to efficientnetB0_100x100.h5


D:\Applications\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/150
99/99 [==============================] - 339s 3s/step - loss: 0.4113 - accuracy: 0.8697 - val_loss: 2.8875 - val_accuracy: 0.1075

Epoch 00002: val_loss improved from 4.21131 to 2.88747, saving model to efficientnetB0_100x100.h5
Epoch 3/150
99/99 [==============================] - 318s 3s/step - loss: 0.3059 - accuracy: 0.9074 - val_loss: 3.1156 - val_accuracy: 0.1011

Epoch 00003: val_loss did not improve from 2.88747
Epoch 4/150
99/99 [==============================] - 321s 3s/step - loss: 0.2104 - accuracy: 0.9333 - val_loss: 2.8627 - val_accuracy: 0.0967

Epoch 00004: val_loss improved from 2.88747 to 2.86269, saving model to efficientnetB0_100x100.h5
Epoch 5/150
99/99 [==============================] - 350s 4s/step - loss: 0.1748 - accuracy: 0.9469 - val_loss: 5.2346 - val_accuracy: 0.1005

Epoch 00005: val_loss did not improve from 2.86269
Epoch 6/150
99/99 [==============================] - 475s 5s/step - loss: 0.1934 - accuracy: 0.9378 - val_loss: 2.5964 - val_accur

In [31]:
%%time
score = model.evaluate(x_train, y_train, batch_size = 64)
print('training score: ', score)

score = model.evaluate(x_val, y_val, batch_size = 64)
print('validation score: ', score)

score = model.evaluate(x_test, y_test, batch_size = 64)
print('test score: ', score)

99/99 [==============================] - 87s 876ms/step - loss: 0.0016 - accuracy: 1.0000
training score:  [0.0015850483905524015, 1.0]
25/25 [==============================] - 19s 770ms/step - loss: 0.1509 - accuracy: 0.9599
validation score:  [0.15094713866710663, 0.9599236845970154]
14/14 [==============================] - 11s 762ms/step - loss: 0.1683 - accuracy: 0.9657
test score:  [0.16826903820037842, 0.9656750559806824]
Wall time: 2min 28s


## Model 2

In [37]:
#Defining the model
base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=x_train.shape[1:],classes=y_train.shape[1])

#Adding the final layers to the above base models where the actual classification is done in the dense layers

model= Sequential()
model.add(base_model) 
model.add(Flatten()) 

#Model summary
model.summary()

#Adding the Dense layers along with activation and batch normalization
#model.add(Dense(1024,activation=('relu'),input_dim=512))
#model.add(Dropout(.5))
#model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
#model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model.add(Dense(10,activation=('softmax'))) 

#Checking the final model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 4, 4, 1280)        4049571   
_________________________________________________________________
flatten_3 (Flatten)          (None, 20480)             0         
Total params: 4,049,571
Trainable params: 4,007,548
Non-trainable params: 42,023
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 4, 4, 1280)        4049571   
_________________________________________________________________
flatten_3 (Flatten)          (None, 20480)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               5243136   
_________________

In [38]:
%%time
#Defining the parameters
batch_size= 64
epochs=150
lr=.005
MAX_PATIENT=5

opt = tf.optimizers.Adam(lr)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('efficientnetB0-2_100x100.h5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=MAX_PATIENT)
ReduceLR = ReduceLROnPlateau(monitpr = 'val_loss',factor=0.1,patience=3, verbose=1)
callbacks_list = [checkpoint,ReduceLR, early]

print("[INFO] Training Start---------please Wait")
history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs, 
                    validation_data =(x_val, y_val),
                    verbose = 1,
                   callbacks = callbacks_list)
print("[INFO] Training Finished!!!")

[INFO] Training Start---------please Wait
Epoch 1/150
99/99 [==============================] - 417s 4s/step - loss: 1.6138 - accuracy: 0.5465 - val_loss: 4.3414 - val_accuracy: 0.1221

Epoch 00001: val_loss improved from inf to 4.34139, saving model to efficientnetB0-2_100x100.h5
Epoch 2/150
99/99 [==============================] - 347s 4s/step - loss: 0.8063 - accuracy: 0.7486 - val_loss: 11.6950 - val_accuracy: 0.0483

Epoch 00002: val_loss did not improve from 4.34139
Epoch 3/150
99/99 [==============================] - 322s 3s/step - loss: 0.6656 - accuracy: 0.7954 - val_loss: 3.0136 - val_accuracy: 0.0585

Epoch 00003: val_loss improved from 4.34139 to 3.01357, saving model to efficientnetB0-2_100x100.h5
Epoch 4/150
99/99 [==============================] - 336s 3s/step - loss: 0.4870 - accuracy: 0.8541 - val_loss: 5.5597 - val_accuracy: 0.1005

Epoch 00004: val_loss did not improve from 3.01357
Epoch 5/150
99/99 [==============================] - 310s 3s/step - loss: 0.4124 - accu

In [ ]:
%%time
score = model.evaluate(x_train, y_train, batch_size = 64)
print('training score: ', score)

score = model.evaluate(x_val, y_val, batch_size = 64)
print('validation score: ', score)

score = model.evaluate(x_test, y_test, batch_size = 64)
print('test score: ', score)

64/99 [==================>...........] - ETA: 30s - loss: 0.0099 - accuracy: 0.9980

In [ ]:
import matplotlib.pyplot as plt
#Plotting the training and validation loss

f,ax=plt.subplots(2,1) #Creates 2 subplots under 1 column

#Assigning the first subplot to graph training loss and validation loss
ax[0].plot(history.history['loss'],color='b',label='Training Loss')
ax[0].plot(history.history['val_loss'],color='r',label='Validation Loss')

#Plotting the training accuracy and validation accuracy
ax[1].plot(history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()